# meta clas

[ref](https://www.notion.so/py-fe2b1096bac845b58555ddca930a5a5a)

劫持了Foo类的创建过程（在创建类的时候做了一些自己的操作）
1. 劫持class创建过程
1. 修改class的内容
1. 返回修改过的class

In [53]:
class Mymeta(type):
    def __init__(self, name, bases, dic): # (cls_name, bases_tuple, attrs_dict)
        super().__init__(name, bases, dic)
        print('===>Mymeta.__init__')
        print(self.__name__)
        print("dict:　", dic)
        print(self.yaml_tag)

    def __new__(cls, *args, **kwargs):
        print('===>Mymeta.__new__')
        print("cls.__name__:", cls.__name__)
        return type.__new__(cls, *args, **kwargs)

    def __call__(cls, *args, **kwargs):
        print('===>Mymeta.__call__')
        print(cls, *args, **kwargs)
        obj = cls.__new__(cls)
        cls.__init__(cls, *args, **kwargs)
        return obj

In [54]:
class Foo(metaclass=Mymeta):
    yaml_tag = '!Foo'

    def __init__(self, name):
        print('Foo.__init__')
        self.name = name

    def __new__(cls, *args, **kwargs):
        print('Foo.__new__')
        return object.__new__(cls)
    
    def print_name(self):
        print(self.name)

# create a class name `Foo`

===>Mymeta.__new__
cls.__name__: Mymeta
===>Mymeta.__init__
Foo
dict:　 {'__module__': '__main__', '__qualname__': 'Foo', 'yaml_tag': '!Foo', '__init__': <function Foo.__init__ at 0x0000020CBFE30670>, '__new__': <function Foo.__new__ at 0x0000020CBFE2FF70>, 'print_name': <function Foo.print_name at 0x0000020CBFE2F550>}
!Foo


In [55]:
foo = Foo("jack")

===>Mymeta.__call__
<class '__main__.Foo'> jack
Foo.__new__
Foo.__init__


In [45]:
foo.print_name()

jack


In [18]:
foo.yaml_tag

'!Foo'

# `type`
The type type, as the default metaclass in Python, defines special methods that new metaclasses can override to implement unique code generation behavior. Here is a brief overview of these "magic" methods that exist on a metaclass:

1. `__new__`: 产生一个实例类的时候调用This method is called on the Metaclass before an instance of a class based on the metaclass is created
1. `__init__`: 在创建实例类之后，用于设置实例类的一些属性 This method is called to set up values after the instance/object is created
1. `__prepare__`: Defines the class namespace in a mapping that stores the attributes
1. `__call__`: 当实例类创建对象的时候使用   This method is called when the constructor of the new class is to be used to create an object

In [12]:
# hello_metaclass.py
# A simple metaclass
# This metaclass adds a 'hello' method to classes that use the metaclass
# meaning, those classes get a 'hello' method with no extra effort
# the metaclass takes care of the code generation for us
class HelloMeta(type):
    # A hello method
    def hello(cls):
        print("greetings from %s, a HelloMeta type class" % (type(cls())))

    # Call the metaclass
    def __call__(self, *args, **kwargs): # cap:影响实例创建过程
        # create the new class as normal
        cls = type.__call__(self, *args)
        print("HelloMeta.__call__(), cls={}".format(cls))

        # define a new hello method for each of these classes
        setattr(cls, "hello", self.hello)

        # return the class
        return cls

# Try out the metaclass
class TryHello(object, metaclass=HelloMeta):
    def greet(self):
        self.hello()

# Create an instance of the metaclass. It should automatically have a hello method
# even though one is not defined manually in the class
# in other words, it is added for us by the metaclass
greeter = TryHello()
# greeter.greet()

HelloMeta.__call__(), cls=<__main__.TryHello object at 0x0000020CBFDFC940>


In [14]:
greeter.greet()

HelloMeta.__call__(), cls=<__main__.TryHello object at 0x0000020CBFDFC220>
greetings from <class '__main__.TryHello'>, a HelloMeta type class


In [24]:
#为什么实例类的实例调用方法的时候，metaclass.__call__方法也会被调用
greeter.greet() 

HelloMeta.__call__(), cls=<__main__.TryHello object at 0x0000020CBFC51580>
greetings from <class '__main__.TryHello'>, a HelloMeta type class


In [13]:
TryHello().greet()

HelloMeta.__call__(), cls=<__main__.TryHello object at 0x0000020CBFDFCBE0>
HelloMeta.__call__(), cls=<__main__.TryHello object at 0x0000020CBFDFC160>
greetings from <class '__main__.TryHello'>, a HelloMeta type class


In [7]:
stu_cls = type("student", (), {"name": "jack"})

In [9]:
a_stu = stu_cls()

In [25]:
a_stu.name

'jack'